Census data is download from Kaggle (it seems census data is not accessible from my location): https://www.kaggle.com/datasets/cdeotte/census-data-for-godaddy?select=ACSST5Y2017.S0101-Column-Metadata.csv
And then processed to make it usable for the model

In [1]:
import json
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from library.utils import read_df, states, states_abb, write_df
from sklearn.feature_selection import r_regression

In [2]:
df_train = read_df('train.csv')

In [3]:
new_col_pop_total = "census_population_x1000"
new_col_pop_over_18 = "census_over_18_population_x1000"


def read_census_population_data(file_name, sub_folder="kaggle_census"):
    # Read file
    df_t = read_df(file_name, sub_folder, skiprows=1)

    original_col_pop_over_18 = (
        "Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!18 years and over"
    )
    original_col_pop_total = "Estimate!!Total!!Total population"

    # Rename columns
    df_t = df_t[
        ["Geographic Area Name", original_col_pop_total, original_col_pop_over_18]
    ].rename(
        columns={
            original_col_pop_over_18: new_col_pop_over_18,
            original_col_pop_total: new_col_pop_total,
        }
    )
    df_t["year"] = int(file_name.split(".")[0].replace("ACSST5Y", "")) + 2

    t1 = df_t.apply(
        lambda x: x["Geographic Area Name"].split(", "),
        axis=1,
    )

    # County and state into separate columns
    df_t[["county", "state"]] = pd.DataFrame(
        t1.tolist(), index=t1.index, columns=["county", "state"]
    )

    # Convert unints to x1000
    df_t[[new_col_pop_total, new_col_pop_over_18]] = (
        df_t[[new_col_pop_total, new_col_pop_over_18]] / 1000
    )
    df_t = df_t.drop(columns=["Geographic Area Name"])
    df_t["state_abb"] = df_t.apply(lambda x: states[x["state"]], axis=1)

    df_t["county"] = df_t["county"].replace("Doña Ana County", "Dona Ana County")

    return df_t

In [4]:
df_census_population = pd.concat(
    [
        read_census_population_data(x)
        for x in os.listdir("../data/kaggle_census/")
        if "Data" in x
    ]
).reset_index(drop=True)
write_df(df_census_population, 'df_census_population.csv', 'kaggle_census')

/mnt/e/data/godaddy-microbusiness-density-forecasting/notebooks/library/utils.py:10: DtypeWarning: Columns (20,21,24,25,172,173,176,177,327,328,479,480,627,628,631,632,780,781,784,785) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(data_path, sub_folder, filename), delimiter=delimiter, skiprows=skiprows)


In [8]:
t1 = pd.merge(
    df_train, df_census_population, "left", ["state", "county", "year", "state_abb"]
)
assert t1[t1['census_population_x1000'].isna()].shape[0] == 0, 'Shape has to be 0'

t2 = t1[['row_id', new_col_pop_total, new_col_pop_over_18]]
write_df(t2, 'df_train_census.csv', 'kaggle_census')

In [37]:
t_2023 = df_census_population[df_census_population["year"] == 2023]
t_2023 = t_2023[
    [
        "county",
        "state",
        "state_abb",
        "census_over_18_population_x1000",
        "census_population_x1000",
    ]
].rename(
    columns={
        "census_over_18_population_x1000": "target_census_over_18_population_x1000",
        "census_population_x1000": "target_census_population_x1000",
    }
)

df_target_population = pd.merge(
    df_train, t_2023, "left", ["state", "county", "state_abb"]
)[
    [
        "row_id",
        "target_census_over_18_population_x1000",
        "target_census_population_x1000",
    ]
]
t2 = pd.merge(df_train, df_target_population, "left", "row_id")
t2["adjusted_microbusiness_density"] = (
    t2["active"] / (t2["target_census_over_18_population_x1000"] * 1000)
) * 100
df_adjusted_microbusiness_density = t2[["row_id", "adjusted_microbusiness_density"]]
write_df(
    df_adjusted_microbusiness_density,
    "df_adjusted_microbusiness_density.csv",
    "kaggle_census",
)

# Many columns which are not used

In [17]:
files = [x for x in os.listdir("../data/kaggle_census/") if "Data" in x]
df_t = read_df(files[0], "kaggle_census")

/mnt/e/data/godaddy-microbusiness-density-forecasting/notebooks/library/utils.py:10: DtypeWarning: Columns (2,3,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,30,33,34,35,36,37,38,39,40,41,42,45,46,47,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,82,83,86,89,90,91,94,95,98,99,100,101,102,103,106,107,108,109,110,111,112,113,114,115,116,117,118,121,122,123,126,127,128,129,130,131,134,135,138,139,142,143,158,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,181,182,185,186,187,188,189,190,191,192,193,194,195,198,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,234,235,238,239,242,243,246,249,250,251,252,253,254,255,258,259,260,261,262,263,264,265,266,267,268,269,270,273,274,277,298,301,302,303,306,309,310,313,314,315,316,317,318,319,320,321,322,325,326,327,328,329,330,333,334,335,338,339,340,341,342,343,344,345,346,347,350,351,354,355,356,

In [18]:
cols = [x for x in list(df_t.columns) if 'margin' not in x.lower()]
cols = [x for x in cols if 'annotation' not in x.lower()]
np.array(cols)

array(['GEO_ID', 'NAME', 'S0101_C01_001E', 'S0101_C01_001EA',
       'S0101_C01_001M', 'S0101_C01_001MA', 'S0101_C01_002E',
       'S0101_C01_002EA', 'S0101_C01_002M', 'S0101_C01_002MA',
       'S0101_C01_003E', 'S0101_C01_003M', 'S0101_C01_003MA',
       'S0101_C01_003EA', 'S0101_C01_004E', 'S0101_C01_004M',
       'S0101_C01_004MA', 'S0101_C01_004EA', 'S0101_C01_005E',
       'S0101_C01_005EA', 'S0101_C01_005M', 'S0101_C01_005MA',
       'S0101_C01_006E', 'S0101_C01_006EA', 'S0101_C01_006M',
       'S0101_C01_006MA', 'S0101_C01_007E', 'S0101_C01_007EA',
       'S0101_C01_007M', 'S0101_C01_007MA', 'S0101_C01_008E',
       'S0101_C01_008M', 'S0101_C01_008MA', 'S0101_C01_008EA',
       'S0101_C01_009E', 'S0101_C01_009M', 'S0101_C01_009MA',
       'S0101_C01_009EA', 'S0101_C01_010E', 'S0101_C01_010M',
       'S0101_C01_010MA', 'S0101_C01_010EA', 'S0101_C01_011E',
       'S0101_C01_011M', 'S0101_C01_011MA', 'S0101_C01_011EA',
       'S0101_C01_012E', 'S0101_C01_012EA', 'S0101_C01_012M',
 

In [41]:
for col in annotation_cols:
    if df_t[~df_t[col].isna()].shape[0] != 0:
        if df_t[col].value_counts().index[0] != '(X)':
            print(df_t[~df_t[col].isna()].shape[0], col)
            break